___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://glasswing.vc/wp-content/uploads/2023/10/16-CNNs.webp" width="350px" height="180px" />


# <font color= #8A0829> Laboratorio de Modelado de Datos </font>
#### <font color= #2E9AFE> `Martes y Viernes (Videoconferencia) de 13:00 - 15:00 hrs`</font>
- <Strong> Sara Eugenia Rodríguez </Strong>
- <Strong> Año </Strong>: 2025
- <Strong> Email: </Strong>  <font color="blue"> `sara.rodriguezr@iteso.mx` </font>
___

<p style="text-align:right;"> Imagen recuperada de: https://glasswing.vc/wp-content/uploads/2023/10/16-CNNs.webp</p>

### <font color= #2E9AFE> Tema: Entrenando un clasificador CNN</font>


**MLP**

Funcionan mejor para:
- Datos tabulares (columnas de características)
- Pequeños conjuntos de datos donde la estructura espacial es irrelevante
- Tareas simples de clasificación o regresión PERO que los datos sean de alta dimensión o que haya relaciones no lineales complejas

No son buenas para:
- Imágenes
- Video
- Audio
- Cualquier cosa con estructuras espaciales

**CNN**

Diseñadas específicamente para imágenes o cualquier cosa que tenga patrones espaciales locales.

Funcionan mejor para:
- Clasificacion de imágenes
- Detección de objetos
- Video

Las CNN parten de tres supuestos:
1. Existe estructura espacial

Una CNN espera que los datos tengan forma (altura, largo, ancho) pero en un dataset tabular como: edad, ingresos, historial crediticio, ciudad. No existe ninguna estructura espacial. No tiene sentido aplicar un filtro 3x3 sobre [Edad, Ingresos, Ciudad]

2. Esperan traslación invariance

Una CNN aprende que un borde sigue siendo un borde, esté donde esté la imagen. Pero en los datos tabulares:
- que "edad" esté en la columna 0 y "ingresos" en columna 1 NO significa que moverlos de sitio conserve el significado. 

No son buenas para:
- Datos tabulares


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple as nt
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.datasets import cifar10

#Creamos un objeto con sus atributos
Data = nt("Data", "x_train y_train x_valid y_valid x_test y_test")

In [ ]:
# Cargamos los datos
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
data = Data(x_train, y_train, None, None, x_test, y_test)

In [ ]:
cifar10_labels = [
    "airplane", 
    "automobile", 
    "bird", 
    "cat", 
    "deer", 
    "dog", 
    "frog", 
    "horse", 
    "ship", 
    "truck"
]

In [ ]:
def visualize_data(data):
    images_to_show = 36
    per_row = 12
    fig = plt.figure(figsize=(20,5))
    for i in range(images_to_show):
        pos = (i // per_row, ((i % per_row) + per_row) % per_row)
        ax = plt.subplot2grid((int(images_to_show / per_row), per_row),
                              pos, xticks=[], yticks=[])
        ax.imshow(np.squeeze(data.x_train[i]))
    plt.show()

In [ ]:
# Visualizamos los datos
visualize_data(data)

In [ ]:
#Vamos a trabjar solo con imagenes de Perros, Ranas y Caballos para que la carga computacional sea menor
selected_classes = [5, 6, 7]

train_mask = np.isin(y_train, selected_classes).flatten()
x_train_filtered = x_train[train_mask]
y_train_filtered = y_train[train_mask]

test_mask = np.isin(y_test, selected_classes).flatten()
x_test_filtered = x_test[test_mask]
y_test_filtered = y_test[test_mask]

filtered_data = Data(
    x_train_filtered,
    y_train_filtered,
    None, None,
    x_test_filtered,
    y_test_filtered
)

visualize_data(filtered_data)

In [ ]:
#Están balanceadas nuestras clases?
classes, counts = np.unique(filtered_data.y_train, return_counts=True)
print(list(zip(classes, counts)))

In [ ]:
# Curva de aprendizaje
def visualize_training(hist):
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epochs')
    plt.legend(['training', 'validation'], loc='lower right')
    plt.show()
    
    # training vs validation loss
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(['training', 'validation'], loc='upper right')
    plt.show()

**Preprocesamiento**

Las imagenes son de 32x32 pixeles con valores entre 0 y 255 por cada pixel. 
Al hacerse demasiadas multiplicaciones vamos a dividir nuestros valores entre 255 de forma que los calculos no exploten en cada iteracion. 

RGB significa:
- R = Red (Rojo)
- G = Green (Verde)
- B = Blue (Azul)

Por ejemplo un pixel:
[255, 0, 0] → rojo puro  
[0, 255, 0] → verde puro  
[0, 0, 255] → azul puro  
[255, 255, 255] → blanco  
[0, 0, 0] → negro

Entonces un pixel puede tener 256 × 256 × 256 ≈ 16 millones de colores, por eso los modelos de deep learning normalizan los valores de 0 a 1. 


De la libreria de Keras vamos a usar la funcion "to_categorical" para hacer el one hot encoding a nuestras etiquetas. 

Vamos a seleccionar las primeras 5000 imagenes como train y las siguientes 5000 como validation

Vamos a usar un set de validation para medir el accuracy durante el entrenamiento de la red yvamos a usar los datos del test para correr toda la red contra datos que no se han visto


In [ ]:

def preprocess(data, categories):
    x_train = data.x_train.astype("float32") / 255
    x_test = data.x_test.astype("float32") / 255
    y_train = to_categorical(data.y_train, categories)
    y_test = to_categorical(data.y_test, categories)    
    return Data(x_train[5000:], y_train[5000:],
                x_train[:5000], y_train[:5000],
                x_test, y_test)

In [ ]:
# Procesamos los datos
label_map = {5: 0, 6: 1, 7: 2}

y_train_mapped = np.vectorize(label_map.get)(filtered_data.y_train)
y_test_mapped  = np.vectorize(label_map.get)(filtered_data.y_test)

filtered_data = Data(
    filtered_data.x_train,
    y_train_mapped,
    filtered_data.x_valid,
    filtered_data.y_valid,
    filtered_data.x_test,
    y_test_mapped
)

categories = len(np.unique(y_train_mapped))
print("Shape de x_train pre-processing: ", filtered_data.x_train.shape)
print("Shape de y_train pre-processing: ", filtered_data.y_train.shape)
processed_data = preprocess(filtered_data, categories)
print("Shape de x_train post-processing: ", processed_data.x_train.shape)
print("Shape de y_train post-processing: ", processed_data.y_train.shape)

## MLP


In [ ]:
def build_mlp(data, categories):
    model = Sequential()
    #En CIFAR una imagen tiene forma 32x32x3 --> 3072 valores, la mlp solo puede procesar vectores
    model.add(Flatten(input_shape=data.x_train.shape[1:]))
    model.add(Dense(1000, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(categories, activation="softmax"))
    
    # Compilamos el modelo
    model.compile(loss="categorical_crossentropy", optimizer="adam", 
                  metrics=["accuracy"])
    return model

In [ ]:
# Build mlp
mlp = build_mlp(processed_data, categories)
print("Arquitectura de la MLP:")
mlp.summary()

Son muchisimos parametros!

In [ ]:
#Entrenamos el mlp
hist_mlp = mlp.fit(processed_data.x_train, processed_data.y_train, batch_size=32, 
                   epochs=20, validation_data=(processed_data.x_valid,
                                                processed_data.y_valid),
                    shuffle=True)

In [ ]:
visualize_training(hist_mlp)

## CNN

In [ ]:
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
def build_cnn(data, categories):
    model = Sequential()
    #algo comun que se hace es duplicar los fitlros en cada bloque
    #padding = same hace que las dimensiones de las imagenes permanezcan controladas
    model.add(Conv2D(filters=16, kernel_size=2, padding="same", activation="relu",
                     input_shape=data.x_train.shape[1:]))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32, kernel_size=2, padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64, kernel_size=2, padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(500, activation="relu"))
    model.add(Dropout(0.4))
    model.add(Dense(categories, activation="softmax"))
    
    # Compile the model
    model.compile(loss="categorical_crossentropy", optimizer="adam",
                  metrics=["accuracy"])
    return model

In [ ]:
cnn = build_cnn(processed_data, categories)
print("CNN architecture:")
cnn.summary()

In [ ]:
# Train the cnn
hist_cnn = cnn.fit(processed_data.x_train, processed_data.y_train, batch_size=32, 
                   epochs=20, validation_data=(processed_data.x_valid, 
                                               processed_data.y_valid))

In [ ]:
visualize_training(hist_cnn)

In [ ]:
# Comparamos los scores
score_mlp = mlp.evaluate(processed_data.x_test, processed_data.y_test, verbose=0)
score_cnn = cnn.evaluate(processed_data.x_test, processed_data.y_test, verbose=0)

print("Accuracy mlp: {0:.2f}%".format(score_mlp[1] * 100))
print("Accuracy cnn: {0:.2f}%".format(score_cnn[1] * 100))

In [ ]:
class_names = {0: "dog", 1: "frog", 2: "horse"}
def visualize_predictions(model, data, class_names, num_images=20):
    # Seleccionamos un subset aleatorio
    idxs = np.random.choice(len(data.x_test), num_images, replace=False)
    images = data.x_test[idxs]
    true_labels = np.argmax(data.y_test[idxs], axis=1)

    # Predicciones
    preds = model.predict(images)
    pred_labels = np.argmax(preds, axis=1)

    plt.figure(figsize=(15, 6))
    for i, idx in enumerate(idxs):
        plt.subplot(4, 5, i+1)
        plt.imshow(images[i])
        plt.axis("off")

        true_c = true_labels[i]
        pred_c = pred_labels[i]

        # Color del texto
        color = "green" if true_c == pred_c else "red"

        plt.title(
            f"Pred: {class_names[pred_c]}\nTrue: {class_names[true_c]}",
            color=color,
            fontsize=10
        )

    plt.tight_layout()
    plt.show()

visualize_predictions(cnn, processed_data, class_names, num_images=20)